## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv
from sklearn.compose import ColumnTransformer
%matplotlib inline


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)       
pd.set_option('display.max_colwidth', None)   


## Criação do DataFrame

In [ ]:
load_dotenv() 
database_url = os.getenv("NEON_DATABASE_URL")

engine = create_engine(database_url)
query = "SELECT * FROM hotel_bookings"

df = pd.read_sql(query, engine)
df.head()

### EDA

"Preprocessing includes lumping infrequent categories of the categorical predictor
Country (originally with 126 levels or countries) into 11 levels (CN, DEU, ESP, FRA, GBR,
IRL, NLD, PRT, USA, NULL and OTHER). The categorical predictor ReservedRoomType has
11 levels, and the categorical predictor AssignedRoomType has 10 levels; the levels of these
two predictors were lumped into a total of seven categories to increase the counts of
infrequent levels."

In [ ]:
def preprocess_hotel_categories(df):
    """
    Replica o pré-processamento do paper:
    - Country: 126 níveis -> 11 níveis (CN, DEU, ESP, FRA, GBR, IRL, NLD, PRT, USA, NULL, OTHER)
    - ReservedRoomType (11 níveis) e AssignedRoomType (10 níveis) -> 7 categorias totais combinadas
    """
    
    df_processed = df.copy()
    
    priority_countries = ['CN', 'DEU', 'ESP', 'FRA', 'GBR', 'IRL', 'NLD', 'PRT', 'USA']
    
    def lump_country(country):
        if pd.isna(country):
            return 'NULL'
        elif country in priority_countries:
            return country
        else:
            return 'OTHER'
    
    df_processed['country'] = df_processed['country'].apply(lump_country)
    
    common_room_types = ['A', 'B', 'C', 'D', 'E', 'F']  # Baseado em datasets típicos
    
    def lump_room_type(room):
        if pd.isna(room):
            return 'NULL'
        elif room in common_room_types:
            return room
        else:
            return 'OTHER'
    
    df_processed['reserved_room_type'] = df_processed['reserved_room_type'].apply(lump_room_type)
    df_processed['assigned_room_type'] = df_processed['assigned_room_type'].apply(lump_room_type)
    
    return df_processed

# APLICAR O PRÉ-PROCESSAMENTO
print("=== ANTES DO PRÉ-PROCESSAMENTO ===")
print("Country níveis únicos:", df['country'].nunique(), df['country'].unique()[:10])
print("ReservedRoomType níveis únicos:", df['reserved_room_type'].nunique())
print("AssignedRoomType níveis únicos:", df['assigned_room_type'].nunique())

# Executar pré-processamento
df = preprocess_hotel_categories(df)

print("\n=== DEPOIS DO PRÉ-PROCESSAMENTO ===")
print("Country níveis únicos:", df['country'].nunique(), sorted(df['country'].unique()))
print("ReservedRoomType níveis únicos:", df['reserved_room_type'].nunique(), sorted(df['reserved_room_type'].unique()))
print("AssignedRoomType níveis únicos:", df['assigned_room_type'].nunique(), sorted(df['assigned_room_type'].unique()))

print("\nDistribuição Country (top 11):")
print(df['country'].value_counts().head(11))


### Pre Processamento

In [ ]:
df['arrival_date_month'] = pd.Categorical(df['arrival_date_month'], 
                                          categories=["January","February","March",
                                                      "April","May","June","July","August",
                                                      "September","October","November","December"], 
                                          ordered=True)

df['arrival_date_day_of_month'] = df['arrival_date_day_of_month'].astype(int)


### Divisão de Treino e Teste
    "75% training set and a 25% test set to estimate the performance of the machine learning algorithms"

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from xgboost import XGBClassifier
from sklearn import svm
from sklearn.model_selection import GridSearchCV

In [ ]:
X = df.drop(columns=['is_canceled'])
y = df['is_canceled']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")

### Normalização

In [ ]:
# 1. DIAGNÓSTICO: Identificar qual coluna tem datas
print("🔍 COLUNAS COM MUITAS CATEGORIAS ÚNICAS:")
for col in X_train.select_dtypes(include=['object']).columns:
    unique_count = X_train[col].nunique()
    if unique_count > 20:  # Suspeita de datas
        print(f"❌ {col}: {unique_count} níveis únicos")
        print(f"   Amostra: {X_train[col].unique()[:5]}")
        print()

# 2. TRATAR COLUNAS DE DATA PROBLEMÁTICAS
def fix_date_columns(df):
    df_fixed = df.copy()
    
    # Converter colunas suspeitas de data para numéricas ou remover
    date_suspect_cols = []
    for col in df_fixed.select_dtypes(include=['object']).columns:
        if df_fixed[col].nunique() > 20:  # Muitas categorias = provavelmente data
            try:
                # Tentar converter para datetime e extrair features
                pd.to_datetime(df_fixed[col], errors='coerce')
                print(f"📅 Convertendo {col} para numérico...")
                df_fixed[col] = pd.to_datetime(df_fixed[col], errors='coerce').dt.dayofyear
            except:
                # Se não conseguir, remover a coluna
                print(f"🗑️ Removendo {col} (demasiadas categorias)")
                df_fixed = df_fixed.drop(columns=[col])
    
    return df_fixed

# APLICAR TRATAMENTO
X_train_fixed = fix_date_columns(X_train)
X_test_fixed = fix_date_columns(X_test)

# 3. AGORA O PREPROCESSADOR FUNCIONA
continuous_cols = X_train_fixed.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X_train_fixed.select_dtypes(include=['object']).columns.tolist()

print(f"\n✅ Após tratamento:")
print("Contínuas:", continuous_cols)
print("Categóricas:", categorical_cols)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), continuous_cols),
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_cols)
    ])

# FIT E TRANSFORM
X_train_processed = preprocessor.fit_transform(X_train_fixed)
X_test_processed = preprocessor.transform(X_test_fixed)

print(f"\n✅ X_train processado: {X_train_processed.shape}")
print(f"✅ X_test processado: {X_test_processed.shape}")


### Treinamento dos Modelos e Otimização dos hiperparâmtros

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

numeric_cols = X_train.select_dtypes(include=[np.number]).columns
X_train_numeric = X_train[numeric_cols]
X_test_numeric = X_test[numeric_cols]


rf_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

rf_grid = GridSearchCV(
    estimator=RandomForestClassifier(),
    param_grid=rf_param_grid,
    cv=5,
    n_jobs=-1
)

rf_grid.fit(X_train_numeric, y_train)
print(f"✅ Melhores parâmetros RandomForest: {rf_grid.best_params_}")
print(f"✅ Score CV: {rf_grid.best_score_:.3f}")



#### XGBoost

In [ ]:
numeric_cols = X_train.select_dtypes(include=[np.number]).columns
X_train_numeric = X_train[numeric_cols]
X_test_numeric = X_test[numeric_cols]

xgb_param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

xgb_grid = GridSearchCV(
    estimator=XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
    param_grid=xgb_param_grid,
    cv=5,
    n_jobs=-1
)

xgb_grid.fit(X_train_numeric, y_train)
print(f"✅ Melhores parâmetros XGBoost: {xgb_grid.best_params_}")
print(f"✅ Score CV: {xgb_grid.best_score_:.3f}")

#### SVM

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # preencher NaN com média da coluna
    ('scaler', StandardScaler()),                  # escalar dados
    ('svm', SVC(random_state=42))
])

pipeline.fit(X_train_numeric, y_train)
print(f"✅ Score treino: {pipeline.score(X_train_numeric, y_train):.3f}")
print(f"✅ Score teste: {pipeline.score(X_test_numeric, y_test):.3f}")
